In [1]:
from netgen.occ import *
from ngsolve import Mesh

import sys
sys.path.append("../build/rigid_body")
from rigid_body import *

import pythreejs as p3

# import numpy as np
from IPython.display import display
import ipywidgets as widgets
# import time

In [2]:
# set up OCC CAD model
box = Box(Pnt(-0.5,-0.5,-0.5), Pnt(0.5,0.5,0.5))
geo = OCCGeometry(box)
mesh = Mesh(geo.GenerateMesh(maxh=0.5))

In [3]:
# from ngsolve.webgui import Draw
# Draw(mesh)

In [4]:
def extract_vertices(mesh: Mesh):
    "extracts a p3js compatible vertex list from a netgen.occ Mesh"
    combined_vertices = [] # flat array of vertices of all faces
    all_vertices = [vert.point for vert in mesh.vertices] # all available vertices

    # throw all vertices of all faces into combined_vertices
    for face in mesh.faces:
        for vertex in face.vertices:
            point = all_vertices[vertex.nr]
            combined_vertices.append(point)
            
    return np.array(combined_vertices, dtype=np.float32)

In [5]:
view_width = 600
view_height = 400

# set up pythreejs 3d object
original_vertices = extract_vertices(mesh)
buffergeom = p3.BufferGeometry(attributes = {"position" : p3.BufferAttribute(original_vertices, normalized=False)})
buffergeom.exec_three_obj_method("computeVertexNormals") # gives normals for shading
material = p3.MeshPhongMaterial(color='#ff3333', shininess=150, morphTargets=True, side="DoubleSide")
p3mesh = p3.Mesh(buffergeom, material, position=(0,0,0))

# extra scene contents
camera = p3.PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = p3.DirectionalLight(position=[0, 10, 10])
ambient_light = p3.AmbientLight()
grid = p3.GridHelper(500, 500//5, "#2F4F4F", "#2F4F4F")

# set up scene
scene = p3.Scene(children=[p3mesh, camera, key_light, ambient_light, grid])
controller = p3.OrbitControls(controlling=camera)
renderer = p3.Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height, antialias=True) # if performance is bad, try removing antalias=True

In [6]:
# set up physics simulation object
r = RigidBody()
r.setMass(mass_cube())

t = Transformation()
t.setTranslation(0.0,0,0)
t.setRotation(0,2,0.009)
t.setRotation(0,1,0.009)
r.dq=t


In [7]:
# print(r.q)

In [8]:
# play button
play = widgets.Play(
    value=10,
    min=0,
    max=10000,
    step=1,
    interval=100/30,
    description="Press play",
    disabled=False
)


In [9]:
p3mesh.matrixAutoUpdate = False

def update():
    "update function, gets called every timestep"
    r.simulate(5,5)

    p3mesh.matrix=r.q.asTuple()

def reset():
    "resets the simulation on button press"
    # TODO
    print("reset") # note the blue icon in the lower right of jupyter


def observer(x):
    "event handler for clickable buttons"
    # if there is a change in time
    if x["type"] == "change":
        # it might be a reset (or rewind, which is used as an alias to reset)
        if str(x["new"]) == "0" or x["name"] == "repeat":
            reset()
        # or it might be a progress in time
        else:
            pass
            # update()

play.observe(observer)


In [10]:
display(play)
display(renderer)

Play(value=10, description='Press play', interval=3, max=10000)

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

### click-and-drag to rotate, scroll to zoom, right-click-and-drag to move